In [1]:
from gapfinder.feedback import create_roi_vs_contours_images

# get the contents of the poi folder, but only the folder names
poi_folder = f"./output/processed_images"
roi_folder = f"./images/roi_images"
metadata_csv = f"{roi_folder}/roi_metadata.csv"
clear_existing = True

create_roi_vs_contours_images( metadata_csv, roi_folder, poi_folder, clear_existing)

Deleting existing roi_vs_contours folder in ./output/processed_images\0_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\10_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\11_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\1_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\2_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\3_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\4_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\5_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\6_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\7_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\8_otsuOffset
Deleting existing roi_vs_contours folder in ./output/processed_images\9_otsuOffset


In [ ]:
print("hello")